In [55]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import wandb
from deepface import DeepFace
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from wandb.keras import WandbCallback

In [56]:
wandb.init(project="Recognition of Iranian actors")

accuracy,▁
epoch,▁
loss,▁
accuracy,0.0
epoch,0
loss,1.1322


In [57]:
data_list = []

for i in range(1,89):
    img_path = f"picture input\{i}.jpg"
    data = DeepFace.represent(img_path, model_name = 'ArcFace', enforce_detection="false")
    data_list.append(data)


In [58]:
df = pd.DataFrame(data_list)
df['target'] = 1
df.to_csv('actors_list.csv', index=False)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,target
0,-0.268664,0.658981,-0.178734,-0.492479,-0.046718,0.454798,-0.265291,-0.177646,0.099696,-0.052121,...,0.184112,0.524436,-0.915923,0.432692,-0.382436,-0.045325,-0.020876,-0.573894,-1.624661,1
1,-0.379638,0.784054,-0.144992,-0.550940,-0.088381,0.450958,-0.258693,-0.229305,0.183707,-0.079858,...,0.233132,0.614820,-1.103076,0.292081,-0.329377,-0.003798,0.012333,-0.526823,-1.585055,1
2,-0.204989,0.687415,-0.124456,-0.575130,-0.013168,0.371509,-0.233152,-0.150302,0.021146,0.016802,...,0.201060,0.554163,-0.834213,0.457211,-0.265702,-0.073344,-0.054038,-0.447802,-1.261396,1
3,0.066392,0.233479,-0.122591,-0.236882,0.090737,0.075403,0.041393,-0.089817,0.183415,0.002755,...,0.214653,-0.179576,0.051778,0.098001,0.277458,0.064000,0.407475,0.298411,0.242925,1
4,-0.397312,0.932544,-0.185639,-0.677433,-0.045761,0.570632,-0.218073,-0.189786,0.095947,-0.151901,...,0.231074,0.620116,-1.111333,0.521732,-0.522948,-0.007055,0.079369,-0.702242,-1.868766,1


In [59]:
config = wandb.config
config.learning_rate = 0.001

In [60]:
X = df.iloc[0:,0:512].values
Y = df['target'].values

In [61]:
X_train,X_test,Y_train,Y_test = train_test_split(X ,Y ,shuffle=True)

In [62]:
model = tf.keras.models.Sequential([
    Dense(128, input_dim=512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

In [63]:
model.compile(optimizer=tf.keras.optimizers.SGD(config.learning_rate),
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

In [66]:
model.fit(X_train, Y_train, epochs=600, callbacks=[WandbCallback()])

Epoch 1/600
3/3 [==============================] - 0s 5ms/step - loss: 0.0387 - accuracy: 1.0000
Epoch 2/600
3/3 [==============================] - 0s 4ms/step - loss: 0.0385 - accuracy: 1.0000
Epoch 3/600
3/3 [==============================] - 0s 4ms/step - loss: 0.0384 - accuracy: 1.0000
Epoch 4/600
3/3 [==============================] - 0s 4ms/step - loss: 0.0383 - accuracy: 1.0000
Epoch 5/600
3/3 [==============================] - 0s 4ms/step - loss: 0.0382 - accuracy: 1.0000
Epoch 6/600
3/3 [==============================] - 0s 0s/step - loss: 0.0381 - accuracy: 1.0000
Epoch 7/600
3/3 [==============================] - 0s 0s/step - loss: 0.0380 - accuracy: 1.0000
Epoch 8/600
3/3 [==============================] - 0s 4ms/step - loss: 0.0380 - accuracy: 1.0000
Epoch 9/600
3/3 [==============================] - 0s 0s/step - loss: 0.0379 - accuracy: 1.0000
Epoch 10/600
3/3 [==============================] - 0s 4ms/step - loss: 0.0378 - accuracy: 1.0000
Epoch 11/600
3/3 [==============

In [67]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 31ms/step - loss: 0.0085 - accuracy: 1.0000


[0.008548315614461899, 1.0]